# *Grove Water Level Sensor Notebook*

## Overview

The Grove Water Level Sensor is used to determine the height of water in a container.  It measures the presence of water in 5 mm increments to 100 mm (20 points).  It utilizes two ATtiny1616 microcontrollers running firmware to detect the water, which is then read by the PYNQ using the inter-integrated circuit (I2C) bus standard and the standard Grove connector.  The PYNQ board acts as the master, requesting data from either of the ATtiny microcontrollers by targeting their I2C address.  Since there are 20 "water level points," one microcontroller measures the lowest 8 points and the other microcontroller measures the 12 highest points.  The I2C address of the lower microcontroller is 0x77 (or 1110111 in binary), and the I2C address of the high microcontroller is 0x78 (or 1111000 in binary).  

![picture](images/grove_waterlevel_hardware.jpg)

### Helpful Links

[Sensor webpage](https://wiki.seeedstudio.com/Grove-Water-Level-Sensor/)

[PMOD to Grove adapter webpage](https://store.digilentinc.com/pynq-grove-system-add-on-board/)

## Using the Sensor

### 1. Load the base Overlay

In [2]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

### 2. Create the Python class so we can get data from the sensor
 - Since I2C is a very common standard, we *extend* our class from the Pmod_IIC class
 - This class adds some basic value checking and convenience functions specific to the sensor

In [3]:
from pynq.lib.pmod import PMOD_GROVE_G3
from pynq.lib.pmod import PMOD_GROVE_G4
from pynq.lib import Pmod_IIC

class Python_Grove_WaterLevel(Pmod_IIC):
    """This class controls the Grove Water Level Sensor.
    
    This class inherits from the PMODIIC class.
    
    Attributes
    ----------
    iop : _IOP
        The _IOP object returned from the DevMode.
    scl_pin : int
        The SCL pin number.
    sda_pin : int
        The SDA pin number.
    iic_addr : int
        The IIC device address.
    
    """
    def __init__(self, pmod_id, gr_pins, addr): 
        """Return a new instance of a Grove Water Level Sensor object. 
        
        Parameters
        ----------
        pmod_id : int
            The PMOD ID (1, 2) corresponding to (PMODA, PMODB).
        gr_pins : list
            Adapter pins selected.
        addr : int
            I2C addresses of the sensor (0x77, 0x78)
        """
        if gr_pins in [PMOD_GROVE_G3, PMOD_GROVE_G4]:
            [scl_pin,sda_pin] = gr_pins
        else:
            raise ValueError("Valid Grove Pins are on G3 or G4.")
        if addr not in [0x77, 0x78]:
            raise ValueError("Valid I2C addresses are 0x77 or 0x78")
        
        super().__init__(pmod_id, scl_pin, sda_pin, addr)
        
        
    def read(self, num_bytes):
        """If targeting low ATtiny microcontroller, can only read the first 1 to 8 bytes (water level points) 
           If targeting high ATtiny microcontroller, can only read the highest 9 to 20 bytes (water level points)
        
        Parameters
        ----------
        num_bytes : int
            Will perform value checking to make sure we don't try to read too many bytes
        
        Returns
        -------
        list :
            Water level bytes at the requested points
        
        """
        if self.iic_addr == 0x77:
            if num_bytes == 0 or num_bytes > 8:
                raise ValueError("Can only request to get 1-8 water points on low object")
        else:
            if num_bytes == 0 or num_bytes > 12:
                raise ValueError("Can only request to get 1-12 water points on high object")
        self.send([0])
        bytes = self.receive(num_bytes)
        return bytes 

### 3. Create two objects, one to read the lowest water points and another to read the highest water points
 - When the objects are created, the arguments to the class constructor tell the PYNQ we are using PMODB connector on the PYNQ board, we are choosing the G3 connector on the PMOD to Grove adapter board, and the I2C address (0x77 or 0x78)

In [4]:
from pynq.lib.pmod import PMOD_GROVE_G3
wls_hi = Python_Grove_WaterLevel(base.PMODB, PMOD_GROVE_G3, 0x78)
wls_lo = Python_Grove_WaterLevel(base.PMODB, PMOD_GROVE_G3, 0x77)

### 4. Read data from the sensor using the objects we created
 - Since the sensor is capacitive, it returns a byte for each point that contains the value 0-255
 - We can merge the data from the two objects to get a better representation of the data on the sensor

In [5]:
lo_bytes = wls_lo.read(8)
print(f"Low bytes : {lo_bytes}")
hi_bytes = wls_hi.read(12)
print(f"High bytes : {hi_bytes}")
sensor_bytes = lo_bytes + hi_bytes
print(f"Sensor bytes : {sensor_bytes}")

Low bytes : [249, 249, 249, 249, 249, 249, 249, 249]
High bytes : [249, 249, 248, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Sensor bytes : [249, 249, 249, 249, 249, 249, 249, 249, 249, 249, 248, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### Helper function
 - Reading bytes isn't very helpful, let's create a threshold that will tell us if the water point is above that threshold

In [7]:
# Water level for each point will be from 0-255
threshold = 100
# Create a list of "True" and "False" that corresponds if water is above that threshold value
water_present = []
for point in sensor_bytes:
    water_present.append(point > threshold)
print(water_present)
# Create a list of "Y" and "N" that corresponds if water is above that threshold value
water_present = []
for point in sensor_bytes:
    water_present.append("Y" if point > threshold else "N")
print(water_present)
# Create a list of 1 and 0 if water is above that threshold value
water_present = []
for point in sensor_bytes:
    water_present.append(1 if point > threshold else 0)
print(water_present)

[True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False]
['Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### Helper function
 - We built a way to say if water is present or not at a water point, let's print how high water is in mm

In [8]:
water_points = sum(water_present)
water_level_mm = water_points*5
print(f"Water level = {water_level_mm} mm = {water_points} points")

Water level = 55 mm = 11 points
